This is a demo colab for MTME. It assumes you have mt_metrics_eval installed on your runtime, and have downloaded the data onto that machine. Run the cells below in order.

# Preliminaries

In [1]:
import pandas as pd
import os
import numpy as np
folder = "../lit_score/cleaned_benchmark_dataset/"
df_list = pd.read_csv(os.path.join(folder, "LitEval_human_annotated_withmetrics.csv"))

df_list['model'] = df_list.model_x.apply(lambda x: x.split("-")[0].replace(" ", "_")) # remove all spaces in model name
df_list["source_"] = df_list["source"].apply(lambda x: x.replace("\n ", "").replace("\n", "")[:50])
#df_list['model'] = df_list.model.apply(lambda x: x if not "translator" in x else "human")
df_list['source'] = df_list.source.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in source
df_list['tgt'] = df_list.tgt.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in sou
df_list['mqm_score'] = df_list.mqm_score.astype(float)
df_list['rating'] = df_list.rating_x.astype(float)
df_list['rating'] = df_list['rating']-1

In [2]:
# load new data
df = pd.read_csv("/Users/zhangran/Documents/GitHub/lit_score/cleaned_benchmark_dataset/benchmark_dataset_all_src_tgt.csv")
folder = "/Users/zhangran/Documents/GitHub/lit_score/comte_trainer/experiments/xcomet_ranking"

In [3]:
import json
import re
for file in os.listdir(folder):
    if file.endswith(".json"):
        with open(os.path.join(folder, file), "r") as f:
            tmp = json.load(f)
            col = file.replace(".json", "").replace("xcomet_ranking_", "").replace("_checkpoints_checkpoint-", "").replace("xcomet_ranking_", "").replace("referencefree_output_", "")
            df[col] = tmp["segment_scores"]

In [4]:
df_list = pd.concat([df_list, df[df["dataset"] == "LitEval"].iloc[:, 6:].reset_index(drop=True)], axis=1)


In [5]:
df_list.columns

Index(['source', 'tgt', 'pair', 'model_x', 'label_x', 'rating_x', 'reasons_x',
       'severity_x', 'annotator', 'mqm_score', 'source_', 'tgt_', 'released',
       'Unnamed: 0.1', 'index', 'model_y', 'direction_index', 'Unnamed: 0',
       'annotation_id', 'batch_id', 'created_at', 'id', 'label_y', 'lead_time',
       'rating_y', 'reasons_y', 'severity_y', 'text', 'updated_at', 'model_id',
       'answer', 'answer_mqm_literary', 'all-acc-flu-sty-term_literary',
       'gemba_all_literary', 'answer_orig', 'answer_mqm_orig',
       'all-acc-flu-sty-term_orig', 'gemba_all_orig', 'comet_xl', 'comet_xxl',
       'prometheus2', 'prometheus', 'model', 'rating',
       'quar_full_20250320_2116568550', 'quar_xxen_20250320_21051212000',
       'quar_fr_en_20250320_19440812300', 'half_xxen_20250320_16390712500',
       'quar_xxen_20250320_21051212150', 'half_full_20250321_11562622350',
       'half_full_20250321_11562610050', 'half_fr_en_20250320_16162112500',
       'half_xxen_20250320_163907615

In [6]:
metrics = ["mqm_score", "rating", "gemba_all_literary", "gemba_all_orig", "comet_xl", "comet_xxl", "prometheus", 
           'quar_full_20250320_2116568550', 'quar_xxen_20250320_21051212000',
       'quar_fr_en_20250320_19440812300', 'half_xxen_20250320_16390712500',
       'quar_xxen_20250320_21051212150', 'half_full_20250321_11562622350',
       'half_full_20250321_11562610050', 'half_fr_en_20250320_16162112500',
       'half_xxen_20250320_1639076150', 'quar_fr_en_20250320_19440812150',
       'quar_full_20250320_21165612000', 'half_fr_en_20250320_1616213150']

In [7]:
ref = "src"
df_format = df_list.pivot(index=["pair", "source_"], columns=["model"], values="mqm_score").reset_index()
df_format = df_format.merge(df_list[["pair", "source_", "source"]], on=["pair", "source_"], how="left").drop_duplicates(subset=["pair", "source_"], keep="first").reset_index(drop=True)
lst_src = df_format["source"].values
lst_src_ = df_format["source_"].values
lst_pair = df_format["pair"].values
lst_model = df_format.columns[2:-1]

for p, g in df_format.groupby(["pair"]):
    os.makedirs(f"benchmark_dataset/liteval/sources/", exist_ok=True)
    os.makedirs(f"benchmark_dataset/liteval/references/", exist_ok=True)
    g[["source"]].fillna("NaN").to_csv(f"benchmark_dataset/liteval/sources/{p}.txt", sep=" ", index=False, header=False)
    g[["source"]].fillna("NaN").to_csv(f"benchmark_dataset/liteval/references/{p}-{ref}.txt", sep=" ", index=False, header=False)

    os.makedirs(f"benchmark_dataset/liteval/documents/", exist_ok=True)
    pd.DataFrame().to_csv(f"benchmark_dataset/liteval/documents/{p}.docs", sep=" ", index=False, header=False)


df_format = df_list.pivot(index=["pair", "source_"], columns=["model"], values="tgt").reset_index()
df_format = df_format.set_index(["source_", "pair"]).loc[(lst_src_, lst_pair), lst_model]

for p, g in df_format.groupby(["pair"]):
    for model in set(lst_model):
        os.makedirs(f"benchmark_dataset/liteval/system-outputs/{p}/", exist_ok=True)
        g[[model]].fillna("NaN").to_csv(f"benchmark_dataset/liteval/system-outputs/{p}/{model}.txt", index=False, sep=" ", header=False)


for score in metrics:
    df_format = df_list.pivot(index=["pair", "source_"], columns=["model"], values=score).reset_index()
    df_format = df_format.set_index(["source_", "pair"]).loc[(lst_src_, lst_pair), lst_model]
    print(score)
    for p, g in df_format.groupby(["pair"]):
        print(p)
        os.makedirs(f"benchmark_dataset/liteval/metric-scores/{p}/", exist_ok=True)
        os.makedirs(f"benchmark_dataset/liteval/human-scores/", exist_ok=True)
        output_concat = pd.DataFrame()
        for model in set(lst_model):
            tmp = g[[model]]
            tmp["model"] = model
            tmp.columns = ["score", "model"]
            output_concat = pd.concat([output_concat, tmp])
        if not score in ["mqm_score", "rating"]:
            output_concat[["model", "score"]].fillna("NaN").to_csv(f"benchmark_dataset/liteval/metric-scores/{p}/{score}-{ref}.seg.score", index=False, sep=" ", header=False)
            output_concat.groupby("model", as_index=False)["score"].mean()[["model", "score"]].to_csv(f"benchmark_dataset/liteval/metric-scores/{p}/{score}-{ref}.sys.score", index=False, sep=" ", header=False)
        else:
            output_concat[["model", "score"]].fillna("NaN").to_csv(f"benchmark_dataset/liteval/human-scores/{p}.{score}.seg.score", index=False, sep=" ", header=False)
            output_concat.groupby("model", as_index=False)["score"].mean()[["model", "score"]].to_csv(f"benchmark_dataset/liteval/human-scores/{p}.{score}.sys.score", index=False, sep=" ", header=False)



/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:9: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for p, g in df_format.groupby(["pair"]):
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:22: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for p, g in df_format.groupby(["pair"]):
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:32: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for p, g i

mqm_score
de-en
de-zh
en-de
en-zh
rating
de-en
de-zh
en-de
en-zh
gemba_all_literary
de-en
de-zh
en-de
en-zh
gemba_all_orig
de-en


/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

de-zh
en-de
en-zh
comet_xl
de-en
de-zh
en-de
en-zh
comet_xxl
de-en
de-zh
en-de
en-zh
prometheus
de-en
de-zh
en-de
en-zh
quar_full_20250320_2116568550
de-en


/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

...

/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:32: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for p, g in df_format.groupby(["pair"]):
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

half_full_20250321_11562622350
de-en
de-zh
en-de
en-zh
half_full_20250321_11562610050
de-en
de-zh
en-de
en-zh
half_fr_en_20250320_16162112500
de-en
de-zh
en-de
en-zh
half_xxen_20250320_1639076150
de-en
de-zh
en-de
en-zh
quar_fr_en_20250320_19440812150
de-en
de-zh
en-de


/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

en-zh
quar_full_20250320_21165612000
de-en
de-zh
en-de
en-zh
half_fr_en_20250320_1616213150
de-en
de-zh
en-de
en-zh


/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["model"] = model
/var/folders/2n/4742tn7s13l5fcnstlm_d7g40000gn/T/ipykernel_86823/905728320.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [8]:
'''
import pandas as pd
import os
import numpy as np
folder = "../lit_score/cleaned_benchmark_dataset/"
df_list = pd.read_csv(os.path.join(folder, "LitEval_human_annotated_withmetrics.csv"))

df_list['model'] = df_list.model_x.apply(lambda x: x.split("-")[0].replace(" ", "_")) # remove all spaces in model name
#df_list['model'] = df_list.model.apply(lambda x: x if not "translator" in x else "human")
df_list['source'] = df_list.source.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in source
df_list['tgt'] = df_list.tgt.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in sou
df_list['mqm_score'] = df_list.mqm_score.astype(float)
df_list['rating'] = df_list.rating_x.astype(float)
df_list['rating'] = df_list['rating']-1
#convert csv to txt file with space as delimiter, groupby pair, each line for each row
for p, g in df_list.groupby(["pair"]):
    os.makedirs(f"benchmark_dataset/liteval/sources/", exist_ok=True)
    g[["source"]].drop_duplicates().to_csv(f"mt-metrics-eval-v2/liteval/sources/{p}.txt", sep=" ", index=False, header=False)
    
    os.makedirs(f"mt-metrics-eval-v2/liteval/human-scores/{p}", exist_ok=True)
    for score in ["mqm_score", "rating"]:
        g[["model", score]].fillna("NaN").to_csv(f"mt-metrics-eval-v2/liteval/human-scores/{p}/{score}.seg.score", index=False, sep=" ", header=False)
        g.groupby("model", as_index=False)[score].mean()[["model", score]].to_csv(f"mt-metrics-eval-v2/liteval/human-scores/{p}/{score}.sys.score", index=False, sep=" ", header=False)

    for score in ["gemba_all_literary", "gemba_all_orig", "comet_xl", "comet_xxl"]:
        os.makedirs(f"mt-metrics-eval-v2/liteval/metric-scores/{p}", exist_ok=True)
        g[score] = g[score].astype(float)
        g[["model", score]].fillna("NaN").to_csv(f"mt-metrics-eval-v2/liteval/metric-scores/{p}/{score}-src.seg.score", index=False, sep=" ", header=False)
        g.groupby("model", as_index=False)[score].mean()[["model", score]].to_csv(f"mt-metrics-eval-v2/liteval/metric-scores/{p}/{score}-src.sys.score", index=False, sep=" ", header=False)

    for model in g.model.unique():
        os.makedirs(f"mt-metrics-eval-v2/liteval/system-outputs/{p}/", exist_ok=True)
        g[g.model == model][["tgt"]].to_csv(f"mt-metrics-eval-v2/liteval/system-outputs/{p}/{model}.txt", index=False, sep=" ", header=False)
'''




'\nimport pandas as pd\nimport os\nimport numpy as np\nfolder = "../lit_score/cleaned_benchmark_dataset/"\ndf_list = pd.read_csv(os.path.join(folder, "LitEval_human_annotated_withmetrics.csv"))\n\ndf_list[\'model\'] = df_list.model_x.apply(lambda x: x.split("-")[0].replace(" ", "_")) # remove all spaces in model name\n#df_list[\'model\'] = df_list.model.apply(lambda x: x if not "translator" in x else "human")\ndf_list[\'source\'] = df_list.source.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in source\ndf_list[\'tgt\'] = df_list.tgt.apply(lambda x: x.replace("\n", "  ")).fillna("") # remove all spaces in sou\ndf_list[\'mqm_score\'] = df_list.mqm_score.astype(float)\ndf_list[\'rating\'] = df_list.rating_x.astype(float)\ndf_list[\'rating\'] = df_list[\'rating\']-1\n#convert csv to txt file with space as delimiter, groupby pair, each line for each row\nfor p, g in df_list.groupby(["pair"]):\n    os.makedirs(f"benchmark_dataset/liteval/sources/", exist_ok=True)\n   

In [9]:
# @title Imports

import numpy as np
import scipy.stats
from pathlib import Path
import sys
# Setup COMET path
COMET_ROOT = Path("mt_metrics_eval")
sys.path.append(str(COMET_ROOT))
from mt_metrics_eval import meta_info
from mt_metrics_eval import data
from mt_metrics_eval import stats
from mt_metrics_eval import tasks

In [10]:
# @title Print all available evalsets

for testset in meta_info.DATA: # dictionary
  print(f'{testset}:', ' '.join(lp for lp in meta_info.DATA[testset]))


wmt24pp: en-ar_EG en-ar_SA en-bg_BG en-bn_IN en-ca_ES en-cs_CZ en-da_DK en-de_DE en-el_GR en-es_MX en-et_EE en-fa_IR en-fi_FI en-fil_PH en-fr_CA en-fr_FR en-gu_IN en-he_IL en-hi_IN en-hr_HR en-hu_HU en-id_ID en-is_IS en-it_IT en-ja_JP en-kn_IN en-ko_KR en-lt_LT en-lv_LV en-ml_IN en-mr_IN en-nl_NL en-no_NO en-pa_IN en-pl_PL en-pt_BR en-pt_PT en-ro_RO en-ru_RU en-sk_SK en-sl_SI en-sr_RS en-sv_SE en-sw_KE en-sw_TZ en-ta_IN en-te_IN en-th_TH en-tr_TR en-uk_UA en-ur_PK en-vi_VN en-zh_CN en-zh_TW en-zu_ZA
wmt24: en-de en-es ja-zh cs-uk en-cs en-hi en-is en-ja en-ru en-uk en-zh
wmt23.sent: en-de
wmt23: en-de he-en zh-en cs-uk de-en en-cs en-he en-ja en-ru en-uk en-zh ja-en ru-en uk-en
wmt22: en-de en-ru zh-en cs-en cs-uk de-en de-fr en-cs en-hr en-ja en-liv en-uk en-zh fr-de ja-en liv-en ru-en ru-sah sah-ru uk-cs uk-en
wmt21.news: en-cs en-de en-ha en-is en-ja en-ru en-zh cs-en de-en de-fr fr-de ha-en is-en ja-en ru-en zh-en
wmt21.tedtalks: en-de en-ru zh-en
wmt21.flores: bn-hi hi-bn xh-zu zu

In [11]:
'''
import os
import tarfile
import urllib.request

TGZ = 'https://storage.googleapis.com/mt-metrics-eval/mt-metrics-eval-v2.tgz'

def _CopyTgz(dest):
  with urllib.request.urlopen(TGZ) as f, open(dest, 'wb') as out:
    out.write(f.read())

def Download():
  """Download database into LocalDir()."""
  path = "./"
  os.makedirs(path, exist_ok=True)
  local_tgz = os.path.join(path, os.path.basename(TGZ))
  _CopyTgz(local_tgz)
  with tarfile.open(local_tgz, 'r:*') as tar:
    tar.extractall(path)

Download()
'''


'\nimport os\nimport tarfile\nimport urllib.request\n\nTGZ = \'https://storage.googleapis.com/mt-metrics-eval/mt-metrics-eval-v2.tgz\'\n\ndef _CopyTgz(dest):\n  with urllib.request.urlopen(TGZ) as f, open(dest, \'wb\') as out:\n    out.write(f.read())\n\ndef Download():\n  """Download database into LocalDir()."""\n  path = "./"\n  os.makedirs(path, exist_ok=True)\n  local_tgz = os.path.join(path, os.path.basename(TGZ))\n  _CopyTgz(local_tgz)\n  with tarfile.open(local_tgz, \'r:*\') as tar:\n    tar.extractall(path)\n\nDownload()\n'

In [12]:
# @title Load data for WMT21 language pairs scored with MQM

all_evs = {}  # name/lp -> evs
for testset in meta_info.DATA:
  if not testset.startswith('liteval'): continue
  for lp in meta_info.DATA[testset]:
    if 'mqm_score' in meta_info.DATA[testset][lp].std_gold.values():
      all_evs[f'{testset}/{lp}'] = data.EvalSet(testset, lp, True, path="./benchmark_dataset")

print('\n'.join(all_evs.keys()))

liteval/de-en
liteval/en-de
liteval/en-zh
liteval/de-zh


In [13]:
# @title Print summaries for all loaded evalsets

print(f'{"name":<20}  segs sys metrics gold  refs std')
for name, evs in all_evs.items():
  nsegs = len(evs.src)
  nsys = len(evs.sys_names)
  nmetrics = len(evs.metric_basenames)
  gold = evs.StdHumanScoreName('sys')
  nrefs = len(evs.ref_names)
  std_ref = evs.std_ref

  print(f'{name:<20} {nsegs:5d} {nsys:3d} {nmetrics:7d} '
        f'{gold:5} {nrefs:4d} {std_ref}')

name                  segs sys metrics gold  refs std
liteval/de-en           46  12      17 mqm_score    0 src
liteval/en-de           47  12      17 mqm_score    0 src
liteval/en-zh           49  12      17 mqm_score    0 src
liteval/de-zh           46  12      17 mqm_score    0 src


# Comparing metrics

In [14]:
# @title Set up for comparing metrics

# There are many different ways to evaluate the performance of MT metrics. The
# most obvious question is what correlation statistic we should use to capture
# the similarity between a vector of metric scores and a vector of gold scores
# (human ratings). A less obvious question is where those vectors come from.
# We'll defer the choice of correlation statistic to later cells, and begin
# by setting some parameters that precisely define the vectors we're interested
# in comparing.

# Use all evalsets that we've loaded.
evs_list = all_evs.values()

# Choose the version of each metric that uses the standard reference for each
# evalset.
main_refs = [{evs.std_ref} for evs in evs_list]
print(main_refs)

# Some alternative references are known to be close to the standard reference.
# Don't include these among systems to be scored if we are including 'human'
# systems. The only currently known instance is refB in wmt21.news/en-de,
# which is similar to the standard refC.
close_refs = [{'refB'} if k == 'wmt21.news/en-de' else set() for k in all_evs]

# Include 'human' systems (ie, reference translations) among systems to be
# scored. This can make the task more challenging, since some metrics are
# biased against less literal references.
include_human = True

# Don't include systems considered to be outliers. These are systems that are
# much better or worse than all other systems, so they are easy for all metrics
# to rank correctly).
include_outliers = False

# Use MQM ratings as gold scores rather than the scores provided by the main
# WMT task. Metrics tasks have used MQM for main results since 2021.
gold_name = 'mqm_score'

# Only compare metrics that have been designated as primary submissions. This
# removes metric variants that are similar to each other, and reduces the size
# of the comparison matrix.
primary_metrics = True

# Don't limit the results to a particular domain. In WMT21, domains are treated
# as separate test-sets, so this is a no-op (WMT22 is a different story).
domain = None

# Set the number of resampling runs for determining whether one metric is better
# than another according to the permutation test. We'll use 5 to make the demo
# finish quickly, but at least 1000 is required for stable results.
k = 100

# Set the size of blocks for 'early stopping' checks during resampling. If
# you're using k = 1000, this can speed up the computation, usually with
# only minimal changes to the results.
psd = stats.PermutationSigDiffParams(block_size = 100)

# Set the p-value for deciding wheter metrics are considered to be significantly
# different. Lower values make the test more stringent.
pval = 0.05

[{'src'}, {'src'}, {'src'}, {'src'}]


In [15]:
# @title Evaluate metrics using global accuracy

# Global accuracy, introduced by Kocmi et al (https://arxiv.org/abs/2107.10821)
# is a robust way to evaluate the performance of a metric across many different
# settings. The idea is to count the number of pairwise system rankings where
# the metric agrees with the gold ranking, and micro average this across all
# settings.

# The output shows the rank of each metric's significance cluster, followed
# by its accuracy, and whether it is statistically tied with (=) or better than
# (>) each lower-ranking metric.

'''
ranks, matrix, _, _ = data.CompareMetricsWithGlobalAccuracy(
    evs_list, main_refs, close_refs, include_human, include_outliers,
    gold_name, primary_metrics, domain, k, psd, pval)

data.PrintMetricComparison(ranks, matrix, pval)
'''

'\nranks, matrix, _, _ = data.CompareMetricsWithGlobalAccuracy(\n    evs_list, main_refs, close_refs, include_human, include_outliers,\n    gold_name, primary_metrics, domain, k, psd, pval)\n\ndata.PrintMetricComparison(ranks, matrix, pval)\n'

In [16]:
# @title Evaluate metrics using system-level Pearson correlation

# Pearson correlation measures the degree of linear correspondence between
# metric and gold scores. Computing a single correlation across different
# evalsets isn't a great idea, so the interface forces you to choose a single
# set. We'll pick 'wmt21.news/en-de'. The part of the computation that extracts
# relevant score vectors is factored into a separate step to allow you to
# compute other correlations with these vectors.

# Notice that the ranking is quite different from the accuracy ranking, partly
# because we're using only a subset of the data, and partly because Pearson and
# accuracy measure different things. The ranking also includes two metrics that
# were automatically filtered out of the accuracy ranking because they weren't
# available for all evalsets.
'''
for p in ["en-de", "de-en", "en-zh", "de-zh"]:
    print(p)
    evs = all_evs["liteval/" + p]
    corrs = data.GetCorrelations(
        evs, 'sys', {evs.std_ref}, {'src'}, include_human, include_outliers,
        gold_name, primary_metrics, domain)
    ranks, matrix, _, _ = data.CompareMetrics(
        corrs, scipy.stats.pearsonr, 'none', k, psd, pval)
    data.PrintMetricComparison(ranks, matrix, pval, evs)
'''

'\nfor p in ["en-de", "de-en", "en-zh", "de-zh"]:\n    print(p)\n    evs = all_evs["liteval/" + p]\n    corrs = data.GetCorrelations(\n        evs, \'sys\', {evs.std_ref}, {\'src\'}, include_human, include_outliers,\n        gold_name, primary_metrics, domain)\n    ranks, matrix, _, _ = data.CompareMetrics(\n        corrs, scipy.stats.pearsonr, \'none\', k, psd, pval)\n    data.PrintMetricComparison(ranks, matrix, pval, evs)\n'

In [17]:
# @title Evaluate metrics using segment-level Kendall correlation

# Kendall correlation is similar to pairwise accuracy, except that it is
# normalized differently. The function calls are identical to the previous one,
# except that we set the 'level' parameter to 'seg', and specify Kendall rather
# than Pearson. The value of the 'average_by' parameter also matters here, as it
# specifies how system x segment score matrices get converted into vectors for
# comparison. We will use 'none', which just flattens the matrices.

# The resulting ranking is similar to the ranking from accuracy. One noticeable
# difference is that the significance clusters are smaller because they are
# based on more data (much larger vectors). Notice that BLEU is absent because
# it isn't available at the segment level.

# need to fill na with some score
dct = pd.DataFrame()

for p in ["en-de", "de-en", "en-zh", "de-zh"]:
    print(p)
    evs = all_evs["liteval/" + p]
    corrs = data.GetCorrelations(
        evs, 'seg', {evs.std_ref}, {'src'}, include_human, include_outliers,
        gold_name, primary_metrics, domain)
    ranks, matrix, _, _ = data.CompareMetrics(
        corrs, stats.KendallVariants, 'none', k, psd, pval)

    print(data.PrintMetricComparison(ranks, matrix, pval, evs))
    tmp = pd.DataFrame(ranks).T[[0]]
    tmp["pair"] = p
    tmp["metric"] = "segment-level_Kendall"
    dct = pd.concat([dct, tmp])

en-de
*gemba_all_literary[noref]               1  0.6423456  . = = = = = = = = = = = = = = = =
*gemba_all_orig[noref]                   1  0.6388454  . . = = = = = = = = = = = = = = =
*comet_xxl[noref]                        1  0.5820011  . . . = = = = = = = = = = = = = =
*half_fr_en_20250320_1616213150[noref]   1  0.5546077  . . . . = = = = = = = = = = = = =
*quar_fr_en_20250320_19440812150[noref]  1  0.5260551  . . . . . = = = = = = = = = = = =
*quar_fr_en_20250320_19440812300[noref]  1  0.5260171  . . . . . . = = = = = = = = = = =
*comet_xl[noref]                         1  0.5231552  . . . . . . . = = = = = = = = = =
*quar_full_20250320_21165612000[noref]   1  0.5152407  . . . . . . . . = = = = = = = = =
*quar_full_20250320_2116568550[noref]    1  0.5145442  . . . . . . . . . = = = = = = = =
*half_fr_en_20250320_16162112500[noref]  1  0.5055063  . . . . . . . . . . = = = = = = =
*half_xxen_20250320_1639076150[noref]    1  0.5001058  . . . . . . . . . . . = = = = = =
*quar_xxen_2025

In [18]:
# @title Evaluate metrics using seg-level accuracy with optimized tie threshold.

# This is an implementation of the acc*_eq pairwise ranking accuracy proposed in
# https://arxiv.org/abs/2305.14324. This is similar to global accuracy, but it
# additionally gives metrics credit for predicting ties in gold scores, which
# arise frequently in MQM segment-level data. To avoid bias due to differences
# in scoring precision for different metrics, an optimal threshold for assigning
# ties is automatically computed for each metric and test set.

# For demo purposes we disable significance testing by setting k to 0.
# (Significance testing works but is currently very slow.) Note that the
# optimization procedure uses sampling, so results can change across different
# runs.


for p in ["en-de", "de-en", "en-zh", "de-zh"]:
    print(p)
    evs = all_evs["liteval/" + p]
    corrs = data.GetCorrelations(
        evs, 'seg', {evs.std_ref}, {'src'}, include_human, include_outliers,
        gold_name, primary_metrics, domain)
    ranks, matrix, _, _ = data.CompareMetrics(
        corrs, stats.KendallWithTiesOpt, 'item', 10, psd, pval, variant='acc23',
        sample_rate=0.7)

    print(data.PrintMetricComparison(ranks, matrix, pval, evs))
    tmp = pd.DataFrame(ranks).T[[0]]
    tmp["pair"] = p
    tmp["metric"] = "acc_eq"
    dct = pd.concat([dct, tmp])

en-de
*comet_xxl[noref]                        1  0.6058332  . = = > = = = = = = = = = = = = =
*quar_full_20250320_21165612000[noref]   1  0.5897302  . . = = = = = = = > = = = = = > =
*quar_full_20250320_2116568550[noref]    1  0.5882803  . . . = = = = = = = > = > = = = =
*quar_fr_en_20250320_19440812300[noref]  2  0.5825511  . . . . = = = = = = = = = = = = =
*quar_xxen_20250320_21051212150[noref]   2  0.5813874  . . . . . = = = = = = = = > = = =
*quar_fr_en_20250320_19440812150[noref]  2  0.5780413  . . . . . . = = = = = = = = = = =
*gemba_all_orig[noref]                   2  0.5646628  . . . . . . . = > = = = = = = = =
*comet_xl[noref]                         2  0.5633716  . . . . . . . . = = = = = = = = =
*half_xxen_20250320_16390712500[noref]   3  0.5609779  . . . . . . . . . = = = = = = = =
*quar_xxen_20250320_21051212000[noref]   3  0.5577399  . . . . . . . . . . = = = = = = =
*gemba_all_literary[noref]               3  0.5576767  . . . . . . . . . . . = = = = = =
*half_fr_en_202

In [19]:
dct.reset_index(inplace=True)
dct.columns = ["metric", "score", "pair", "corr"]

In [20]:
dct.groupby(["corr", "metric"], as_index=False)["score"].mean()

,corr,metric,score
0,acc_eq,comet_xl-src,0.527333
1,acc_eq,comet_xxl-src,0.534665
2,acc_eq,gemba_all_literary-src,0.523554
3,acc_eq,gemba_all_orig-src,0.548876
4,acc_eq,half_fr_en_20250320_16162112500-src,0.509983
5,acc_eq,half_fr_en_20250320_1616213150-src,0.338963
6,acc_eq,half_full_20250321_11562610050-src,0.500909
7,acc_eq,half_full_20250321_11562622350-src,0.461513
8,acc_eq,half_xxen_20250320_16390712500-src,0.520527
9,acc_eq,half_xxen_20250320_1639076150-src,0.527107


In [21]:
df_list.model.unique()

array(['translator1', 'qwen2', 'm2m', 'nllb', 'translator3', 'deepl',
       'towerinstruct', 'gpt4o', 'google_translate', 'meta',
       'translator2', 'gemma'], dtype=object)

In [65]:
# check if human scores are ranked higher than system scores for each source
def check_humanvsllm(df_list, metric, source_col = "source_", human_col = ["translator1", "translator2", "translator3"], llm_col = ["gpt4o", "google_translate", "qwen2", "deepl"]):
    lst = []
    for p in df_list["pair"].unique():
        n_humanllm = 0
        n_humanmt = 0
        count = 0
        for source_, g in df_list[df_list["pair"] == p].groupby([source_col]):
            g.drop_duplicates(subset=["model", source_col], inplace=True)
            human_m = set(g["model"]).intersection(set(human_col))
            mt_m = set(g["model"]).difference(human_m)
            human_max = g.set_index(["model"]).loc[list(human_m), metric].max()
            mt_max = g.set_index(["model"]).loc[list(mt_m), metric].max()
            if human_max > mt_max:
                n_humanmt += 1
            if llm_col:
                llm_m = set(g["model"]).intersection(set(llm_col))
                llm_max = g.set_index(["model"]).loc[list(llm_m), metric].max()
                if human_max > llm_max:
                    n_humanllm += 1
            count += 1
        if count > 0:
            lst.append((p, metric, n_humanllm/count, n_humanmt/count))
        else:
            print(source_, g.model.unique())
            print(p, metric, "no human scores")
            break
    return lst


In [66]:
import warnings
warnings.filterwarnings("ignore")
adequacy_df = pd.DataFrame()
for m in metrics:
    print(m)
    tmp = pd.DataFrame(check_humanvsllm(df_list, m), columns=["pair", "metric", "humanllm", "humanmt"])
    adequacy_df = pd.concat([adequacy_df, tmp])

mqm_score
rating
gemba_all_literary
gemba_all_orig
comet_xl
comet_xxl
prometheus
quar_full_20250320_2116568550
quar_xxen_20250320_21051212000
quar_fr_en_20250320_19440812300
half_xxen_20250320_16390712500
quar_xxen_20250320_21051212150
half_full_20250321_11562622350
half_full_20250321_11562610050
half_fr_en_20250320_16162112500
half_xxen_20250320_1639076150
quar_fr_en_20250320_19440812150
quar_full_20250320_21165612000
half_fr_en_20250320_1616213150


In [67]:
adequacy_df.groupby(["metric"], as_index=False)[["humanllm", "humanmt"]].mean()

,metric,humanllm,humanmt
0,comet_xl,0.180993,0.126760
1,comet_xxl,0.259926,0.232983
2,gemba_all_literary,0.069307,0.063872
3,gemba_all_orig,0.059219,0.059219
4,half_fr_en_20250320_16162112500,0.296855,0.227215
5,half_fr_en_20250320_1616213150,0.048797,0.037928
6,half_full_20250321_11562610050,0.143702,0.106005
7,half_full_20250321_11562622350,0.169675,0.089165
8,half_xxen_20250320_16390712500,0.212169,0.195865
9,half_xxen_20250320_1639076150,0.169139,0.136863


In [68]:
# literarytranslation
iyyer = df[df.dataset == "literarytran"]
metrics_ = iyyer.columns.tolist()[-12:]
adequacy_df = pd.DataFrame()
for m in metrics_:
    print(m)
    tmp = pd.DataFrame(check_humanvsllm(iyyer, m, source_col= "src", human_col= ["human"], llm_col=None), columns=["pair", "metric", "humanllm", "humanmt"])
    adequacy_df = pd.concat([adequacy_df, tmp])

quar_full_20250320_2116568550
quar_xxen_20250320_21051212000
quar_fr_en_20250320_19440812300
half_xxen_20250320_16390712500
quar_xxen_20250320_21051212150
half_full_20250321_11562622350
half_full_20250321_11562610050
half_fr_en_20250320_16162112500
half_xxen_20250320_1639076150
quar_fr_en_20250320_19440812150
quar_full_20250320_21165612000
half_fr_en_20250320_1616213150


In [70]:
adequacy_df.groupby(["metric"], as_index=False)[["humanllm", "humanmt"]].mean()

,metric,humanllm,humanmt
0,half_fr_en_20250320_16162112500,0.0,0.216667
1,half_fr_en_20250320_1616213150,0.0,0.083333
2,half_full_20250321_11562610050,0.0,0.100000
3,half_full_20250321_11562622350,0.0,0.116667
4,half_xxen_20250320_16390712500,0.0,0.127778
5,half_xxen_20250320_1639076150,0.0,0.133333
6,quar_fr_en_20250320_19440812150,0.0,0.100000
7,quar_fr_en_20250320_19440812300,0.0,0.100000
8,quar_full_20250320_21165612000,0.0,0.127778
9,quar_full_20250320_2116568550,0.0,0.133333


In [26]:
# @title Evaluate a new metric

# New metrics can be included in the comparison of existing metrics using the
# 'extern_metrics' argument to GetCorrelations(). To demonstrate this, we'll
# create and evaluate a new metric consisting of the average of the top 3
# metrics in the system-level Pearson ranking.

# The result is a slight, non-significant, improvement over C-SPECpn, the metric
# with highest Pearson correlation. (The '*' before the new metric indicates
# that it isn't recognized as a primary submission.)

evs = all_evs['wmt21.news/en-de']

# Create the new metric
top3_metrics = ['C-SPECpn-refC', 'COMET-QE-MQM_2021-src', 'bleurt-20-refC']
sys_scores = {}
for sys_name in evs.sys_names:
  if sys_name == 'refC': continue
  scores = np.array([evs.Scores('sys', m)[sys_name] for m in top3_metrics])
  sys_scores[sys_name] = scores.mean(axis=0)

# Run the comparison with the new metric included via the 'extern_metrics'
# argument.
extras = {'top3_avg-refC': sys_scores}
corrs = data.GetCorrelations(
    evs, 'sys', {evs.std_ref}, {'refB'}, include_human, include_outliers,
    gold_name, primary_metrics, domain, extern_metrics=extras)
ranks, matrix, _, _ = data.CompareMetrics(
    corrs, scipy.stats.pearsonr, 'none', k, psd, pval)

data.PrintMetricComparison(ranks, matrix, pval, evs)

KeyError: 'wmt21.news/en-de'

In [ ]:
# @title Evaluate a new metric using global accuracy

# This requires a bit more work, since we have to produce results for multiple
# evalsets. As before, the result is a slight gain over the best single metric
# (note that the averaged metrics aren't quite the top 3 for the global accuracy
# task).

# Create the new metric, one instance per input evalset
top3_metrics = ['C-SPECpn-<REF>', 'COMET-QE-MQM_2021-src', 'bleurt-20-<REF>']
extras_list = []
for evs in evs_list:
  top3 = [m.replace('<REF>', evs.std_ref) for m in top3_metrics]
  sys_scores = {}
  for sys_name in evs.sys_names:
    if sys_name == evs.std_ref: continue
    scores = np.array([evs.Scores('sys', m)[sys_name] for m in top3])
    sys_scores[sys_name] = scores.mean(axis=0)
  extras_list.append({f'top3_avg-{evs.std_ref}': sys_scores})

# Run the comparison with the new metric included via the 'extern_metrics_list'
# argument.
ranks, matrix, _, _ = data.CompareMetricsWithGlobalAccuracy(
    evs_list, main_refs, close_refs, include_human, include_outliers,
    gold_name, primary_metrics, domain, k, psd, pval,
    extern_metrics_list=extras_list)

data.PrintMetricComparison(ranks, matrix)

# Ranking metrics using the task interface

This is a higher-level interface designed to make it more convenient to compare
a set of metrics using various different criteria called 'tasks'. The following
code uses this interface to roughly duplicate the comparisons in the previous
section.

In [ ]:
# @title Define a set of tasks

# Create TaskSets from dicts that specify attribute/value-list combinations,
# along with fixed assignments to other attributes. Concatenate these into a
# single TaskSet.

k = 1  # Use only a single random draw for demo.
lang0 = {'test_set': ['wmt21.news'], 'lang': ['en-de,en-ru,zh-en']}
langs = {'test_set': ['wmt21.news'], 'lang': ['en-de', 'en-ru', 'zh-en']}

taskset = tasks.TaskSet(
    lang0, corr_fcn='accuracy', close_refs=[{'refB'}, set(), set()], k=k)
taskset += tasks.TaskSet(langs, level='sys', corr_fcn='pearson', k=k)
taskset += tasks.TaskSet(langs, level='seg', corr_fcn='pearson', k=k)
taskset += tasks.TaskSet(
    langs, level='seg', avg_by='item', corr_fcn='KendallWithTiesOpt',
    perm_test='pairs', corr_fcn_args={'sample_rate': 0.1}, k=k)

# A TaskSet is just a list of Tasks, so we can make arbitrary changes to
# attribute values. In this case, set the correct close_refs for en-de tasks.

for task in taskset:
  if task.lang == 'en-de': task.close_refs = {'refB'}

# Print task 'names' (attribute/value strings in canonical order).

for t in taskset:
  print(t.name)

In [ ]:
# @title Run the tasks

# This first loads the necessary data, then runs each task in sequence to
# produce a TaskSetResults object. Subsequent runs re-use the loaded data.

results = taskset.Run()  # Takes about 5 minutes.

In [ ]:
# @title Print raw task results

for result in results:
  print(result.name)
  print(result.Str())

In [ ]:
# @title Average ranks for metrics

# To combine the performance of metrics across tasks, we average their task
# ranks. The tasks are weighted to ensure that the total mass for important
# attributes is evenly distributed among the different values those attributes
# take on.
weights = results.AssignWeights(tasks.Attributes())
global_ranks = results.AverageRanks(weights)

# It is also interesting to compare the metric performance on different subsets
# of tasks, for instance split by language.
ranks_by_lp = {}
for val, subset in results.SplitByAttr('lang').items():
  weights = subset.AssignWeights(tasks.Attributes())
  ranks_by_lp[val] = subset.AverageRanks(weights)

# Print out the comparison, with global ranks first, followed by a breakdown
# by language pair. We only show metrics that are in the intersection of all
# tasks.
langs = [' all ' if lp == 'en-de,en-ru,zh-en' else lp for lp in ranks_by_lp]
print(''.rjust(24), 'global', ' '.join(langs))
for metric, rank in global_ranks.items():
  ranks_for_metric = [rank] + [d[metric] for d in ranks_by_lp.values()]
  print(f'{metric:<25}', ' '.join(f'{r:5.2f}' for r in ranks_for_metric))
